# Exploring Magnetic Reconnection
## Heliophysics Summer School - Magnetic Reconnection Lab
---

**Goals:**


1.   Understand the major features of "fast" magnetic reconnection.
2.   Examine the unique features of reconnection in the dayside and nightside magnetosphere.
3.   Engage with spacecraft measurement methods and reconstruction/identification of reconnection regions.

---

## Introduction

In [ ]:
%pip install -qU "airavata-python-sdk[notebook]"
import airavata_jupyter_magic

%authenticate

# PLEASE NOTE: At a given time, ONLY run a single job in one of the available clusters: Anvil OR Jetstream. 
# Anvil is the default cluster and if you need to run on Jetstream; comment the Anvil and uncomment the Jetstream
# Anvil
%request_runtime test_cpu --file=cybershuttle.yml --group=Gkeyll --walltime=120 --use=AnvilCPU:shared
# Jetstream
# %request_runtime test_cpu --file=cybershuttle.yml --group=Gkeyll --walltime=120 --use=Gkeyll:cloud

%wait_for_runtime test_cpu --live
%switch_runtime test_cpu

In [ ]:
!find ~/cybershuttle/dataset/Plasma-Vlab-amitava-class/ -mindepth 1 -maxdepth 1 -type d -exec ln -s {} . \;

In [ ]:
# Import python packages
import matplotlib
import matplotlib.pyplot as plt
import postgkyl as pg
import numpy as np

In [ ]:
# Helper function for computing vector potential
def calc_psi2d(fx, fy, dx=1, dy=1):#solenoidal flows
	'''
	Calcualte psi by integrating dpsi = -fy*dx + fx*dy, psi[0,0]=0.
	Notes: 
		1. (fx=dpsi/dy,fy=-dpsi/dx) is called Hamiltonian gradient of psi, and	contours of psi give vector field (fx, fy);
		2. div(f)=0
	'''
	ny,nx=fx.shape
	psi=np.zeros((ny,nx))
	for jx in range(1,nx):
		psi[0,jx]=psi[0,jx-1]-fy[0,jx]*dx
	for jy in range(1,ny):
		psi[jy,:]=psi[jy-1,:]+fx[jy,:]*dy
	# since f = rot(A) gives extra restraints on f (e.g., div(f)=0)
	# it makes sense that information provided by fy[1:nx,:] is useless here
	return psi

# Helper function for finding X-points
def find_saddles(mat : np.ndarray) -> list:                                                                                                                              
    (N, M) = mat.shape                                                                     
                                                                                           
    jMax = np.argmax(mat, axis = 1) # index of col for max in each row                     
    iMin = np.argmin(mat, axis = 0) # index of row for min in each col                     
                                                                                           
    IJMax = [(i,jMax[i]) for i in range(N)] # list of indexes of max of each row           
    IJMin = [(iMin[j],j) for j in range(M)] # list of indexes of min of each col           
                                                                                           
    maxRowMinCol = list(set(IJMax) & set(IJMin)) # max of row, min of col                         
                                                                                           
    iMax = np.argmax(mat, axis = 0) # index of row for max in each col                     
    jMin = np.argmin(mat, axis = 1) # index of col for min in each row                     
                                                                                           
    IJMax = [(iMax[j],j) for j in range(M)] # list of indexes of max of each col           
    IJMin = [(i,jMin[i]) for i in range(N)] # list of indexes of min of each row           
                                                                                           
    minRowMaxCol = list(set(IJMax) & set(IJMin)) # min of row, max of col                                                                                          
    
    return maxRowMinCol + minRowMaxCol  

### $\texttt{Gkeyll}$ and $\texttt{postgkyl}$

The code we will be using to model magnetic reconnection is [$\texttt{Gkeyll}$](https://gkeyll.readthedocs.io/), a general purpose simulation framework for a variety of fluid and plasma systems. You can download and install $\texttt{Gkeyll}$ yourself by following the installation instructions on our [Github repo](https://github.com/ammarhakim/gkylzero). 

To read the data, we will utilize the post-processing suite we have developed alongside $\texttt{Gkeyll}$, [$\texttt{postgkyl}$](https://github.com/ammarhakim/postgkyl), which you can also download and install via the instructions on Github. The cluster we will be utilizing for analyzing the results of our simulations already has installations of $\texttt{Gkeyll}$ and $\texttt{postgkyl}$; we already imported postgkyl in this Jupyter Notebook, so if we did not have $\texttt{postgkyl}$, that import command would not have worked! 

The output of $\texttt{Gkeyll}$ simulations can be manipulated in one of two ways: through the GData class, which retains useful metadata from the simulation to subsequent operations, or by directly fetching the raw values and grid and storing them in Numpy arrays for our subsequent manipulations. We will utilize both means of reading the data as we analyze our simulation results. 

In [ ]:
def get_Gdata(pre, frame):
    data_elc = pg.data.GData("%s-elc_%d.gkyl" % (pre, frame))
    data_ion = pg.data.GData("%s-ion_%d.gkyl" % (pre, frame))
    data_field = pg.data.GData("%s-field_%d.gkyl" % (pre, frame))

    return data_elc, data_ion, data_field

In [ ]:
def read_data(data_elc, data_ion, data_field):
    fluid_elc = data_elc.get_values()
    fluid_ion = data_ion.get_values()  
    field = data_field.get_values()  

    # Same grid for all electrons, ions, and EM fields
    coords = data_elc.get_grid()
    # Center the grid values
    for d in range(len(coords)):
        coords[d] = 0.5*(coords[d][:-1] + coords[d][1:])

    return coords, fluid_elc, fluid_ion, field

### The Ten-Moment Multi-Fluid Equations

We first consider the Vlasov-Maxwell system of equations
\begin{align}
  & \frac{\partial f_s}{\partial t} + \nabla \cdot \left (\mathbf{v} f_s \right ) + \nabla_{\mathbf{v}} \cdot \left ( \frac{q_s}{m_s} \left [\mathbf{E} + \mathbf{v} \times \mathbf{B} \right] f_s \right ) = 0 \\
  & \epsilon_0 \mu_0 \frac{\partial \mathbf{E}}{\partial t} - \nabla \times \mathbf{B} = \mu_0 \mathbf{J}, \qquad \mathbf{J} = \sum_s q_s \int \mathbf{v} f_s \thinspace d\mathbf{v} \\
  & \nabla \cdot \mathbf{E} = \frac{\rho_c}{\epsilon_0}, \qquad \rho_c = \sum_s q_s \int f_s \thinspace d\mathbf{v} \\
  & \frac{\partial \mathbf{B}}{\partial t} + \nabla \times \mathbf{E} = 0, \qquad \nabla \cdot \mathbf{B} = 0  
\end{align}

To derive fluid equations from the Vlasov-Maxwell system, we take (mass-weighted) velocity moments of the evolution equation for the distribution function. The simplest sets of fluid equations utilize the zeroth, first, and either scalar second or tensor second moment of the equation. The zeroth moment gives the familiar conservation of mass equation: 
\begin{equation}
  (1)\quad\quad\quad\frac{\partial \rho_s}{\partial t} + \nabla \cdot \left ( \rho_s \mathbf{u}_s \right ) = 0 
\end{equation}
The first moment gives the familiar conservation of momentum equation 
\begin{equation}
  (2)\quad\quad\quad\frac{\partial \rho_s \mathbf{u}_s}{\partial t} + \nabla \cdot \left ( \rho_s \mathbf{u}_s \mathbf{u}_s + \mathbf{P}_s \right ) = \frac{q_s}{m_s} \left ( \rho_s \mathbf{E} + \rho_s \mathbf{u}_s \times \mathbf{B} \right )
\end{equation}
where $\mathbf{P}_s$ is the pressure tensor of species $s$.

Now we consider the tensor second moment of the Vlasov equation, switching to Einstein summation notation for clarity on the coupling to the rank-3 heat-flux tensor: 
\begin{equation}
(3)\quad\quad\quad\frac{\partial \mathcal{P}_{ij_s}}{\partial t} + \frac{\partial \mathcal{Q}_{ijk_s}}{\partial x_k} = \frac{q_s}{m_s} \rho_s u_{s[i} E_{j]} + \frac{q_s}{m_s} \epsilon_{[ikl} \mathcal{P}_{kj_s]} B_l
\end{equation}
where the square brackets around indices represent the minimal sum over permutations of free indices needed to yield completely symmetric tensors. So for example, $u_{s[i} E_{j]} = u_{i_s} E_j + u_{j_s} E_i$. This tensor second moment $\mathcal{P}_{ij_s} = P_{ij_s} + \rho_s u_{i_s} u_{j_s} = \mathbf{P}_s + \rho_s \mathbf{u}_s \mathbf{u}_s$ is often referred to as the stress tensor and includes both the pressure tensor and Reynolds stress. The rank-3 heat-flux tensor can be written in general as
\begin{align}
  \mathcal{Q}_{ijk_s} = Q_{ijk_s} + u_{s[i} \mathcal{P}_{jk_s]} - 2 \rho_s u_{i_s} u_{j_s} u_{k_s}
\end{align}
where
\begin{align}
  Q_{ijk_s} = \int (v_i - u_{i_s}) (v_j - u_{j_s}) (v_k - u_{k_s}) f_s \thinspace d\mathbf{v}
\end{align}
is the plasma rest-frame heat flux. We now see what is commonly referred to as the "closure" problem as we have 10 equations, 1 from the mass density evolution, 3 from the momentum density evolution, and 6 from the stress tensor evolution, but 20 unknowns thanks to the 10 unknown components for the plasma rest-frame heat flux. To close the system, we must express the plasma rest-frame heat flux in terms of lower moments. A straightforward equation we will use expresses 
\begin{align}
  \frac{\partial Q_{ijk_s}}{\partial x_k} & = v_{th_s} k_0 \left (P_{ij_s} - p_s \right ) \\ 
  p_s & = \frac{1}{3} P_{ii}, \qquad v_{th_s} = \sqrt{\frac{p_s}{\rho_s}}.
\end{align}
In other words, the plasma rest-frame heat flux works to relax the pressure tensor back to isotropy and a scalar pressure at a specified length scale $k_0$. 

### Discussion

- How do these equations differ from ideal MHD?

Recall the Generalized Ohm's Law:

\begin{equation}
        \mathbf{E} + \mathbf{u}\times\mathbf{B} = \underbrace{\eta\mathbf{J}}_{\text{Resistivity}} + \underbrace{\frac{\mathbf{J}\times\mathbf{B}}{n_e|e|}}_{\text{Hall}} - \underbrace{\frac{\nabla\cdot\mathbf{P}_e}{n_e|e|}}_{\text{Pressure}} + \frac{m_e}{n|e|^2}\times\underbrace{\left[\frac{\partial\mathbf{J}}{\partial t} + \nabla\cdot\left(\mathbf{u}\mathbf{J} + \mathbf{J}\mathbf{u} - \frac{\mathbf{J}\mathbf{J}}{n|e|}\right)\right]}_{\text{Inertia}}.
\end{equation}

- What fundamental physics important to reconnection is being captured by the ten-moment multi-fluid system that is not captured by MHD?
- What terms, if any, are **not** represented in the ten-moment system?

### Definition

The **agyrotropy** of the pressure tensor is a measure of the strength of the off-diagonal terms in the pressure tensor. Using the definition by Swisdak (2015):
\begin{equation}
A_Q = \sqrt{\frac{P_{xy}^2 + P_{xz}^2 + P_{yz}^2}{P_{\perp}^2 + 2P_{\perp}P_{\parallel}}}.
\end{equation}

## Part 1: Fast Magnetic Reconnection

Magnetic reconnection that occurs in the magnetosphere is "fast"; that is, it occurs at time scales much shorter than the estimates captured by models such as Sweet-Parker and ideal magnetohydrodynamics (MHD). Using ten-moment two-fluid simulation data, we will explore the features of fast reconnection.

### Simulation Parameters

When running simulations, it is often convenient to define a set of normalized units in which to perform the simulation. Many physical constants, such as the permittivity of free space, $\epsilon_0 = 8.85 \times 10^{-12} F/m$, or the actual electron and proton mass, $m_e = 9.11\times 10^{-31} kg, m_p = 1.67\times 10^{-27}kg$ respectively, could lead to enhanced truncation error due to finite precision arithmetic if we carried around the physical values of these constants in our simulations. 

Below we define the units of the electron-ion multi-fluid magnetic reconnection simulations we have performed. 
 
### Discussion
- Review the list of parameters below and discuss the ones you don't understand
- What assumptions are made in choosing these parameters?
- Which parameters are unphysical?


In [ ]:
# Physical constants and derived parameters
epsilon0 = 1.0 # Permittivity of free space.
mu0 = 1.0 # Permeability of free space.
light_speed = 1.0/np.sqrt(epsilon0*mu0) # Speed of light. 
mass_ion = 100.0 # Ion mass.
charge_ion = 1.0 # Ion charge.
mass_elc = 1.0 # Electron mass.
charge_elc = -1.0 # Electron charge.
Ti_over_Te = 5.0 # Ion temperature / electron temperature.
n0 = 1.0 # Reference number density.
nb_over_n0 = 0.2 # Background number density / reference number density.
wpi = np.sqrt(charge_ion**2 * n0 / (epsilon0 * mass_ion)) # Ion plasma frequency. 
wpe = np.sqrt(charge_ion**2 * n0 / (epsilon0 * mass_elc)) # Electron plasma frequency. 
di = light_speed/wpi # Ion inertial length. 
de = light_speed/wpe # Electron inertial length. 
B0 = 0.5 # Reference magnetic field strength. Sets ratio of vAe/c. 
beta = 1.0 # Plasma beta.
omega_ci = charge_ion * B0 / mass_ion # Ion cyclotron frequency. 
vA0 = B0/np.sqrt(mu0*n0*mass_ion) # Reference Alfven speed. 
vA_up = B0/np.sqrt(mu0*nb_over_n0*mass_ion) # Upstream reference Alfven speed from background density. 
Lambda_ee = 2.0e5 # Plasma parameter for computing collision time. 
Ti_frac = Ti_over_Te / (1.0 + Ti_over_Te) # Fraction of total temperature from ions.
Te_frac = 1.0 / (1.0 + Ti_over_Te) # Fraction of total temperature from electrons.
T_tot = beta * (B0 * B0) / 2.0 / n0 # Total temperature.
T_elc_ref = T_tot*Te_frac # Reference electron temperature. 

Time: The time is set by the ion cyclotron frequency.
- **We have written out the data every $\Delta t_{IO} = 0.5 \Omega_{ci}^{-1}$ to time $t_{end} = 30 \Omega_{ci}^{-1}$ for a total of 60 frames**.

In [ ]:
# Frame of simulation data (0-60)
frame = 0

### Reading in Simulation Results

The next cell reads the simulation results.

In [ ]:
# Load data - re-run if changing frame
data_elc, data_ion, data_field = get_Gdata("10m_gem_mass100/rt_10m_gem", frame)
r_coords, r_fluid_elc, r_fluid_ion, r_field = read_data(data_elc, data_ion, data_field)

# Compute out-of-plane current density
Nx = r_fluid_ion.shape[0]
Ny = r_fluid_ion.shape[1]
Jz = np.zeros((Nx, Ny))
Jz = charge_ion/mass_ion*r_fluid_ion[...,3] + charge_elc/mass_elc*r_fluid_elc[...,3]
n_e = r_fluid_elc[...,0]
n_i = r_fluid_ion[...,0]
ux_e = r_fluid_elc[...,1]/r_fluid_elc[...,0]
uy_e = r_fluid_elc[...,2]/r_fluid_elc[...,0]
uz_e = r_fluid_elc[...,3]/r_fluid_elc[...,0]
ux_i = r_fluid_ion[...,1]/r_fluid_ion[...,0]
uy_i = r_fluid_ion[...,2]/r_fluid_ion[...,0]
uz_i = r_fluid_ion[...,3]/r_fluid_ion[...,0]
pxx = r_fluid_elc[:,:,4]
pxy = r_fluid_elc[:,:,5]
pxz = r_fluid_elc[:,:,6]
pyy = r_fluid_elc[:,:,7]
pyz = r_fluid_elc[:,:,8]
pzz = r_fluid_elc[:,:,9]

Pxx = pxx - n_e*ux_e*ux_e
Pxy = pxy - n_e*ux_e*uy_e
Pxz = pxz - n_e*ux_e*uz_e
Pyy = pyy - n_e*uy_e*uy_e
Pyz = pyz - n_e*uy_e*uz_e
Pzz = pzz - n_e*uz_e*uz_e

# Compute contours of the magnetic field.
Ex = r_field[...,0]
Ey = r_field[...,1]
Ez = r_field[...,2]
Bx = r_field[...,3]
By = r_field[...,4]
Bz = r_field[...,5]
dx = r_coords[0][1] - r_coords[0][0]
dy = r_coords[1][1] - r_coords[1][0]
psi = np.zeros((Ny, Nx))
psi = calc_psi2d(Bx.transpose(),By.transpose(), dx, dy)

# Compute magnitude of the magnetic field
_, magB_sq = pg.tools.mag_sq(data_field, '3:6')
magB = np.sqrt(magB_sq[...,0])

bx = Bx/magB
by = By/magB
bz = Bz/magB

# Compute local Alfven speed
_, rho_ion = pg.tools.get_density(data_ion)
vA = magB/np.sqrt(mu0*rho_ion[...,0])

# Compute normalized drift speed
coords, ux =  pg.tools.get_vx(data_ion)
coords, uy = pg.tools.get_vy(data_ion)
uxi_norm0 = ux[...,0]/vAi # Normalized x flow to upstream Alfven speed
uyi_norm0 = uy[...,0]/vAi

coords, ux =  pg.tools.get_vx(data_elc)
coords, uy = pg.tools.get_vy(data_elc)
uxe_norm0 = ux[...,0]/vA_up # Normalized x flow to upstream Alfven speed
uye_norm0 = uy[...,0]/vA_up

udrift = np.sqrt(ux_i**2 + uy_i**2)/vAi

# Compute X-point
saddles = find_saddles(psi)
xpoint = [r_coords[0][saddles[0][1]], r_coords[1][saddles[0][0]]]

# Calculate agyrotropy of the pressure tensor
Ppar = (Pxx*bx**2 + Pyy*by**2 + Pzz*bz**2 + 2.0*(Pxy*bx*by + Pxz*bx*bz + Pyz*by*bz))
Pperp = (Pxx + Pyy + Pzz - Ppar)/2.0
agyro = np.sqrt((Pxy**2 + Pxz**2 + Pyz**2)/(Pperp**2 + 2.0*Pperp*Ppar))

### Plotting different varables
The next cell sets which variables are displayed. 
- The "Color Plot" variables will set the variable represented by the color map.
- The "Streamline" variables set quantities to plot as streamlines/points.

In [ ]:
# Reconnection layer

# Color plots - only activate one!
show_Jz = 1 # Out-of-plane current
show_Bz = 0 # Out-of-plane magnetic field
show_Ez = 0 # Out-of-plane electric field
show_magB = 0 # Magnetic field magnitude
show_vA = 0 # Local Alfven speed
show_ux = 0 # Ion parallel velocity
show_ion_drift = 0 # Ion bulk speed
show_agyro = 0 # Electron pressure agyrotropy
show_n = 0 # Plasma density

# Streamlines
show_psi = 1 # Vector potential
show_ue = 0 # Electron drifts
show_ui = 0 # Ion drifts
show_xpoint = 0 # X-point

In [ ]:
# Plot quantities - re-run to plot new quantities
plt.figure(figsize=(10,5))
# Add shading='gouraud' for further interpolation and smoother plots
if show_Jz == True:
    vmax = np.max(np.abs(Jz))
    vmin = -vmax
    plt.pcolormesh(r_coords[0]/di, r_coords[1]/di, Jz.transpose(), vmax=vmax, vmin=vmin, cmap='seismic')#, shading='gouraud')
    plt.colorbar(format='%.2f', ticks=np.linspace(vmin, vmax, 5), fraction=0.046, pad=0.04)
    plt.title(r'$J_z$')
if show_Bz == True:
    vmax = np.max(np.abs(Bz))
    vmin = -vmax
    plt.pcolormesh(r_coords[0]/di, r_coords[1]/di, Bz.transpose(), vmax=vmax, vmin=vmin, cmap='seismic')#, shading='gouraud')
    plt.colorbar(format='%.2f', ticks=np.linspace(vmin, vmax, 5), fraction=0.046, pad=0.04)
    plt.title(r'$B_z$')
if show_Ez == True:
    vmax = np.max(np.abs(Ez))
    vmin = -vmax
    plt.pcolormesh(r_coords[0]/di, r_coords[1]/di, Ez.transpose(), vmax=vmax, vmin=vmin, cmap='seismic')#, shading='gouraud')
    plt.colorbar(format='%.2f', ticks=np.linspace(vmin, vmax, 5), fraction=0.046, pad=0.04)
    plt.title(r'$E_z$')
if show_magB == True:
    vmax = np.max(np.abs(magB))
    vmin = 0.0
    plt.pcolormesh(r_coords[0]/di, r_coords[1]/di, magB.transpose(), vmax=vmax, vmin=vmin, cmap='inferno')#, shading='gouraud')
    plt.colorbar(format='%.2f', ticks=np.linspace(vmin, vmax, 5), fraction=0.046, pad=0.04)
    plt.title(r'$|\mathbf{B}|$')
if show_vA == True:
    vmax = np.max(np.abs(vA))
    vmin = 0.0
    plt.pcolormesh(r_coords[0]/di, r_coords[1]/di, vA.transpose(), vmax=vmax, vmin=vmin, cmap='inferno')#, shading='gouraud')
    plt.colorbar(format='%.1f', ticks=np.linspace(vmin, vmax, 5), fraction=0.046, pad=0.04)
    plt.title(r'$v_A$')
if show_ux == True:
    vmax = np.max(np.abs(uxi_norm0))
    vmin = -vmax
    plt.pcolormesh(r_coords[0]/di, r_coords[1]/di, uxi_norm0.transpose(), vmax=vmax, vmin=vmin, cmap='seismic')#, shading='gouraud')
    plt.colorbar(format='%.1f', ticks=np.linspace(vmin, vmax, 5), fraction=0.046, pad=0.04)
    plt.title(r'$u_x/v_A$')
if show_ion_drift == True:
    vmax = np.max(np.abs(udrift))
    vmin = 0.0
    plt.pcolormesh(r_coords[0]/di, r_coords[1]/di, udrift.transpose(), vmax=vmax, vmin=vmin, cmap='inferno')#, shading='gouraud')
    plt.colorbar(format='%.1f', ticks=np.linspace(vmin, vmax, 5), fraction=0.046, pad=0.04)
    plt.title(r'$u/v_A$')
if show_agyro == True:
    vmax = np.max(np.abs(agyro))
    vmin = 0.0
    plt.pcolormesh(r_coords[0]/di, r_coords[1]/di, agyro.transpose(), vmax=vmax, vmin=vmin, cmap='inferno')#, shading='gouraud')
    plt.colorbar(format='%.1f', ticks=np.linspace(vmin, vmax, 5), fraction=0.046, pad=0.04)
    plt.title(r'$A_Q$')
if show_n == True:
    vmax = np.max(np.abs(n_e + n_i))
    vmin = 0.0
    plt.pcolormesh(r_coords[0]/di, r_coords[1]/di, (n_e + n_i).transpose(), vmax=vmax, vmin=vmin, cmap='inferno')#, shading='gouraud')
    plt.colorbar(format='%.1f', ticks=np.linspace(vmin, vmax, 5), fraction=0.046, pad=0.04)
    plt.title(r'$n$')
    
X, Y = np.meshgrid(r_coords[0]/di, r_coords[1]/di)
if show_psi == True:
    plt.contour(r_coords[0]/di, r_coords[1]/di, psi, 7, colors="k", linestyles="solid")
if show_ue == True:
    plt.streamplot(X, Y, ux_e.transpose(), uy_e.transpose(), color='green', density=1.0)
if show_ui == True:
    plt.streamplot(X, Y, ux_i.transpose(), uy_i.transpose(), color='purple', density=0.7, broken_streamlines=True)

if show_xpoint == True:
    plt.scatter(xpoint[0]/di, xpoint[1]/di, color='green', marker='X')
    
plt.xlabel(r'$x/d_i$')
plt.ylabel(r'$y/d_i$')
plt.setp(plt.gca(), aspect=1.0)
plt.show()

### Discussion
- Draw what you think the magnetic field lines and velocity lines will look like after a few steps.
- Change a frame number to advance the time and see how the system evolves.
- Try choosing different variables for the color map and streamline display to verify your predictions.
- Where is the diffusion region? What different quantities can you use to identify it?
- Where is the ion inflow stagnation point?
- What happens to the pressure tensor in the diffusion region?

### Profiles across the reconnection layer
We can sample the simulation data in a line running from the top of the simulation to the bottom at x = 0.  
Choose different values below and plot them for different times during the simulation.  

In [ ]:
# Profiles across the reconnection layer
show_Bx = 1
show_Jz = 1
show_uy = 1
show_ne = 0
show_ni = 0

In [ ]:
# Plot at x=0
x0 = int(Nx/2)
plt.figure()
ymax = 0.0
ymin = 0.0
if show_Bx == True:
    plt.plot(r_coords[1]/di, Bx[x0,...], 'k', label=r'$B_x$')
    ymax = max(np.max(np.abs(Bx[x0,...])), ymax)
    ymin = -ymax
if show_Jz == True:
    plt.plot(r_coords[1]/di, Jz[x0,...], 'b', label=r'$J_z$')
    ymax = max(np.max(np.abs(Jz[x0,...])), ymax)
    ymin = -ymax
if show_uy == True:
    plt.plot(r_coords[1]/di, uy_norm0[x0,...], 'r', label=r'$u_y/v_A$')
    ymax = max(np.max(np.abs(uy_norm0[x0,...])), ymax)
    ymin = -ymax
if show_ne == True:
    plt.plot(r_coords[1]/di, n_e[x0,...], 'y', label=r'$n_e$')
    ymax = max(np.max(np.abs(n_e[x0,...])), ymax)
    ymin = min(0.0, ymin)
if show_ni == True:
    plt.plot(r_coords[1]/di, n_i[x0,...], 'r', label=r'$n_i$')
    ymax = max(np.max(np.abs(n_i[x0,...])), ymax)
    ymin = min(0.0, ymin)

plt.xlabel(r'$y/d_i$')
plt.xlim(-6.0, 6.0)
plt.ylim(ymin, ymax)
plt.legend(loc='best')
plt.show()

### Exploration and Discussion
- Explore the different variables available to plot. Are the results consistent with what you would expect?
- Try changing the X value of the line cute through the simulation.  What do you see different?
- Can you identify the region of the outflow jets in these plots? 

### Reconnection Rate

The reconnection rate is the rate of magnetic flux transfer occuring due to reconnection. We can compute this by either evaluating the reconnecting electric field $E_z$ at the X-point, or computing the rate of change of the vector potential.

In [ ]:
# Compute the reconnection rate
def compute_flux(pre, frame): 
    data_elc, data_ion, data_field = get_Gdata(pre, frame)
    r_coords, r_fluid_elc, r_fluid_ion, r_field = read_data(data_elc, data_ion, data_field)
    # Compute vector potential at X point.
    dx = r_coords[0][1] - r_coords[0][0]
    dy = r_coords[1][1] - r_coords[1][0]
    psi = np.zeros((Ny, Nx))
    psi = calc_psi2d(r_field[...,3].transpose(), r_field[...,4].transpose(), dx, dy)
    saddles = find_saddles(psi)
    psiX = psi[saddles[0][0], saddles[0][1]]
    psi0 = np.max(np.abs(psi))
    # Fetch Ez at X point
    EzX = r_field[saddles[0][1], saddles[0][0], 2]
    return psiX, psi0, EzX
    
num_frames = 61
psiX_10m = np.zeros(num_frames)
psi0_10m = np.zeros(num_frames)
EzX_10m = np.zeros(num_frames)
t = np.linspace(0, 30/omega_ci, num_frames)
rate_fac = 1/(B0*vA_up)
for i in range(0, num_frames):
    # Compute the flux at the X-point in 10 moment simulation
    psiX_10m[i], psi0_10m[i], EzX_10m[i] = compute_flux("10m_gem_mass100/rt_10m_gem", i) 

# We can either use Ez or the time derivative of the vector potential psi to estimate the reconnection rate. 
psi_diff_10m = np.abs(psi0_10m - psiX_10m)
dpsi_dt_10m = np.gradient(psi_diff_10m, t)
plt.figure()
plt.plot(t*omega_ci, np.abs(dpsi_dt_10m)*rate_fac, 'k-', label=r'$d\psi/dt$')
plt.plot(t*omega_ci, np.abs(EzX_10m)*rate_fac, 'k--', label=r'$E_z$')
plt.xlabel(r'$t \Omega_{ci}$')
plt.ylabel(r'$E_z/(v_A B_0)$')
plt.legend(loc='best')
plt.show()

## Part 2: Symmetric and Asymmetric Reconnection in the Magnetosphere

## Nightside Reconnection (Magnetotail)

### Activity

Examine the nightside reconnection layer. Compare the features to previous cases, and identify differences.

In [ ]:
# Physical constants and derived parameters
epsilon0 = 1.0 # Permittivity of free space.
mu0 = 1.0 # Permeability of free space.
light_speed = 1.0/np.sqrt(epsilon0*mu0) # Speed of light. 
mass_ion = 1.0 # Ion mass.
charge_ion = 1.0 # Ion charge.
mass_elc = 1.0/256.0 # Electron mass.
charge_elc = -1.0 # Electron charge.
Ti_over_Te = 5.0 # Ion temperature / electron temperature.
n0 = 1.0 # Reference number density.
nb_over_n0 = 0.1 # Background number density / reference number density.
ve = 0.017*light_speed
vA0 = ve*np.sqrt(mass_elc/mass_ion*2.0*(1 + Ti_over_Te))
wpi = np.sqrt(charge_ion**2 * n0 / (epsilon0 * mass_ion)) # Ion plasma frequency. 
wpe = np.sqrt(charge_ion**2 * n0 / (epsilon0 * mass_elc)) # Electron plasma frequency. 
di = light_speed/wpi # Ion inertial length. 
de = light_speed/wpe # Electron inertial length. 
B0 = vA0*np.sqrt(mu0*n0*mass_ion) # Reference magnetic field strength. Sets ratio of vAe/c. 
Bg = 0.1*B0
beta = 1.0 # Plasma beta.
omega_ci = charge_ion * B0 / mass_ion # Ion cyclotron frequency. 
Ti_frac = Ti_over_Te / (1.0 + Ti_over_Te) # Fraction of total temperature from ions.
Te_frac = 1.0 / (1.0 + Ti_over_Te) # Fraction of total temperature from electrons.
T_tot = beta * (B0 * B0) / 2.0 / n0 # Total temperature.
T_elc_ref = T_tot*Te_frac # Reference electron temperature. 

In [ ]:
# Frame of simulation data (0-25)
frame = 0

In [ ]:
# Load data - re-run if changing frame
data_elc, data_ion, data_field = get_Gdata("10m_magnetotail/10m_magnetotail", frame)
r_coords, r_fluid_elc, r_fluid_ion, r_field = read_data(data_elc, data_ion, data_field)

# Compute out-of-plane current density
Nx = r_fluid_ion.shape[0]
Ny = r_fluid_ion.shape[1]
Jz = np.zeros((Nx, Ny))
Jz = charge_ion/mass_ion*r_fluid_ion[...,3] + charge_elc/mass_elc*r_fluid_elc[...,3]
n_e = r_fluid_elc[...,0]
n_i = r_fluid_ion[...,0]
ux_e = r_fluid_elc[...,1]/r_fluid_elc[...,0]
uy_e = r_fluid_elc[...,2]/r_fluid_elc[...,0]
uz_e = r_fluid_elc[...,3]/r_fluid_elc[...,0]
ux_i = r_fluid_ion[...,1]/r_fluid_ion[...,0]
uy_i = r_fluid_ion[...,2]/r_fluid_ion[...,0]
uz_i = r_fluid_ion[...,3]/r_fluid_ion[...,0]
pxx = r_fluid_elc[:,:,4]
pxy = r_fluid_elc[:,:,5]
pxz = r_fluid_elc[:,:,6]
pyy = r_fluid_elc[:,:,7]
pyz = r_fluid_elc[:,:,8]
pzz = r_fluid_elc[:,:,9]

Pxx = pxx - n_e*ux_e*ux_e
Pxy = pxy - n_e*ux_e*uy_e
Pxz = pxz - n_e*ux_e*uz_e
Pyy = pyy - n_e*uy_e*uy_e
Pyz = pyz - n_e*uy_e*uz_e
Pzz = pzz - n_e*uz_e*uz_e

# Compute contours of the magnetic field.
Ex = r_field[...,0]
Ey = r_field[...,1]
Ez = r_field[...,2]
Bx = r_field[...,3]
By = r_field[...,4]
Bz = r_field[...,5]
dx = r_coords[0][1] - r_coords[0][0]
dy = r_coords[1][1] - r_coords[1][0]
psi = np.zeros((Ny, Nx))
psi = calc_psi2d(Bx.transpose(),By.transpose(), dx, dy)

# Compute magnitude of the magnetic field
_, magB_sq = pg.tools.mag_sq(data_field, '3:6')
magB = np.sqrt(magB_sq[...,0])

bx = Bx/magB
by = By/magB
bz = Bz/magB

# Compute magnitude of the magnetic field
_, magB_sq = pg.tools.mag_sq(data_field, '3:6')
magB = np.sqrt(magB_sq[...,0])

# Compute local Alfven speed
_, rho_ion = pg.tools.get_density(data_ion)
vA = magB/np.sqrt(mu0*rho_ion[...,0])

# Compute normalized drift speed
coords, ux =  pg.tools.get_vx(data_ion)
coords, uy = pg.tools.get_vy(data_ion)
ux_norm0 = ux[...,0]/vA0 # Normalized x flow to upstream Alfven speed
uy_norm0 = uy[...,0]/vA0

udrift = np.sqrt(ux_i**2 + uy_i**2)/vAi

# Compute X-point
saddles1 = find_saddles(psi)
xpoint1 = [coords[0][saddles1[1][1]], coords[1][saddles1[1][0]]]
saddles2 = find_saddles(psi[:,:len(coords[0])//2])
xpoint2 = [coords[0][saddles2[1][1]], coords[1][saddles2[1][0]]]

# Compute ion stagnation point
vel_i = np.sqrt(ux_i[np.shape(ux_i)[0]//4:3*np.shape(ux_i)[0]//4,np.shape(ux_i)[1]//4:3*np.shape(ux_i)[1]//4]**2 + uy_i[np.shape(ux_i)[0]//4:3*np.shape(ux_i)[0]//4,np.shape(ux_i)[1]//4:3*np.shape(ux_i)[1]//4]**2)
vel_coordsx = r_coords[0][np.shape(ux_i)[0]//4:3*np.shape(ux_i)[0]//4]
vel_coordsy = r_coords[1][np.shape(ux_i)[1]//4:3*np.shape(ux_i)[1]//4]
ion_stag = [vel_coordsx[np.unravel_index(np.argmin(vel_i, axis=None), vel_i.shape)[0]], vel_coordsy[np.unravel_index(np.argmin(vel_i, axis=None), vel_i.shape)[1]]]

# Calculate agyrotropy of the pressure tensor
Ppar = (Pxx*bx**2 + Pyy*by**2 + Pzz*bz**2 + 2.0*(Pxy*bx*by + Pxz*bx*bz + Pyz*by*bz))
Pperp = (Pxx + Pyy + Pzz - Ppar)/2.0
agyro = np.sqrt((Pxy**2 + Pxz**2 + Pyz**2)/(Pperp**2 + 2.0*Pperp*Ppar))

In [ ]:
# Reconnection layer

# Color plots - only activate one!
show_Jz = 1 # Out-of-plane current
show_Bz = 0 # Out-of-plane magnetic field
show_Ez = 0 # Out-of-plane electric field
show_magB = 0 # Magnetic field magnitude
show_vA = 0 # Alfven speed
show_ux = 0 # Ion parallel velocity
show_ion_drift = 0 # Ion bulk speed
show_agyro = 0 # Electron pressure agyrotropy
show_n = 0 # Plasma density

# Streamlines
show_psi = 1 # Vector potential
show_ue = 0 # Electron drifts
show_ui = 0 # Ion drifts
show_xpoint = 0 # X-point

In [ ]:
# Plot quantities - re-run to plot new quantities
plt.figure(figsize=(15,3))
# Add shading='gouraud' for further interpolation and smoother plots
if show_Jz == True:
    vmax = np.max(np.abs(Jz))
    vmin = -vmax
    plt.pcolormesh(r_coords[0]/di, r_coords[1]/di, Jz.transpose(), vmax=vmax, vmin=vmin, cmap='seismic')#, shading='gouraud')
    plt.colorbar(format='%.2f', ticks=np.linspace(vmin, vmax, 5), fraction=0.046, pad=0.04)
    plt.title(r'$J_z$')
if show_Ez == True:
    vmax = np.max(np.abs(Ez))
    vmin = -vmax
    plt.pcolormesh(r_coords[0]/di, r_coords[1]/di, Ez.transpose(), vmax=vmax, vmin=vmin, cmap='seismic')#, shading='gouraud')
    plt.colorbar(format='%.2f', ticks=np.linspace(vmin, vmax, 5), fraction=0.046, pad=0.04)
    plt.title(r'$E_z$')
if show_Bz == True:
    vmax = np.max(np.abs(Bz))
    vmin = -vmax
    plt.pcolormesh(r_coords[0]/di, r_coords[1]/di, Bz.transpose(), vmax=vmax, vmin=vmin, cmap='seismic')#, shading='gouraud')
    plt.colorbar(format='%.2f', ticks=np.linspace(vmin, vmax, 5), fraction=0.046, pad=0.04)
    plt.title(r'$B_z$')
if show_magB == True:
    vmax = np.max(np.abs(magB))
    vmin = 0.0
    plt.pcolormesh(r_coords[0]/di, r_coords[1]/di, magB.transpose(), vmax=vmax, vmin=vmin, cmap='inferno')#, shading='gouraud')
    plt.colorbar(format='%.2f', ticks=np.linspace(vmin, vmax, 5), fraction=0.046, pad=0.04)
    plt.title(r'$|\mathbf{B}|$')
if show_vA == True:
    vmax = np.max(np.abs(vA))
    vmin = 0.0
    plt.pcolormesh(r_coords[0]/di, r_coords[1]/di, vA.transpose(), vmax=vmax, vmin=vmin, cmap='inferno')#, shading='gouraud')
    plt.colorbar(format='%.1f', ticks=np.linspace(vmin, vmax, 5), fraction=0.046, pad=0.04)
    plt.title(r'$v_A$')
if show_ux == True:
    vmax = np.max(np.abs(uxi_norm0))
    vmin = -vmax
    plt.pcolormesh(r_coords[0]/di, r_coords[1]/di, uxi_norm0.transpose(), vmax=vmax, vmin=vmin, cmap='seismic')#, shading='gouraud')
    plt.colorbar(format='%.1f', ticks=np.linspace(vmin, vmax, 5), fraction=0.046, pad=0.04)
    plt.title(r'$u_x/v_A$')
if show_ion_drift == True:
    vmax = np.max(np.abs(udrift))
    vmin = 0.0
    plt.pcolormesh(r_coords[0]/di, r_coords[1]/di, udrift.transpose(), vmax=vmax, vmin=vmin, cmap='inferno')#, shading='gouraud')
    plt.colorbar(format='%.1f', ticks=np.linspace(vmin, vmax, 5), fraction=0.046, pad=0.04)
    plt.title(r'$u/v_A$')
if show_agyro == True:
    vmax = np.max(np.abs(agyro))
    vmin = 0.0
    plt.pcolormesh(r_coords[0]/di, r_coords[1]/di, agyro.transpose(), vmax=vmax, vmin=vmin, cmap='inferno')#, shading='gouraud')
    plt.colorbar(format='%.1f', ticks=np.linspace(vmin, vmax, 5), fraction=0.046, pad=0.04)
    plt.title(r'$\sqrt{Q}$')
if show_n == True:
    vmax = np.max(np.abs(n_e + n_i))
    vmin = 0.0
    plt.pcolormesh(r_coords[0]/di, r_coords[1]/di, (n_e + n_i).transpose(), vmax=vmax, vmin=vmin, cmap='inferno')#, shading='gouraud')
    plt.colorbar(format='%.1f', ticks=np.linspace(vmin, vmax, 5), fraction=0.046, pad=0.04)
    plt.title(r'$n$')
    
X, Y = np.meshgrid(r_coords[0]/di, r_coords[1]/di)
if show_psi == True:
    plt.contour(r_coords[0]/di, r_coords[1]/di, psi, 7, colors="k", linestyles="solid")
if show_ue == True:
    plt.streamplot(X, Y, ux_e.transpose(), uy_e.transpose(), color='green', density=1.0)
if show_ui == True:
    plt.streamplot(X, Y, ux_i.transpose(), uy_i.transpose(), color='purple', density=0.7, broken_streamlines=True)

if show_xpoint == True:
    plt.scatter(xpoint1[0]/di, xpoint1[1]/di, color='green', marker='X')
    plt.scatter(xpoint2[0]/di, xpoint2[1]/di, color='green', marker='X')

plt.xlabel(r'$x/d_i$')
plt.ylabel(r'$y/d_i$')
plt.setp(plt.gca(), aspect=1.0)
plt.show()

### Discussion

- Is the island in the center physical?
- What might we expect to happen to the island in an actual tail reconnection event?
- How does this relate to geomagnetic activity?

In [ ]:
# Profiles across the reconnection layer
show_Bx = 1
show_Jz = 1
show_uy = 1
show_ne = 0
show_ni = 0

In [ ]:
# Plot at x=0
x0 = int(Nx/2)
plt.figure()
ymax = 0.0
ymin = 0.0
if show_Bx == True:
    plt.plot(r_coords[1]/di, Bx[x0,...], 'k', label=r'$B_x$')
    ymax = max(np.max(np.abs(Bx[x0,...])), ymax)
    ymin = -ymax
if show_Jz == True:
    plt.plot(r_coords[1]/di, Jz[x0,...], 'b', label=r'$J_z$')
    ymax = max(np.max(np.abs(Jz[x0,...])), ymax)
    ymin = -ymax
if show_uy == True:
    plt.plot(r_coords[1]/di, uy_norm0[x0,...], 'r', label=r'$u_y/v_A$')
    ymax = max(np.max(np.abs(uy_norm0[x0,...])), ymax)
    ymin = -ymax
if show_ne == True:
    plt.plot(r_coords[1]/di, n_e[x0,...], 'y', label=r'$n_e$')
    ymax = max(np.max(np.abs(n_e[x0,...])), ymax)
    ymin = min(0.0, ymin)
if show_ni == True:
    plt.plot(r_coords[1]/di, n_i[x0,...], 'r', label=r'$n_i$')
    ymax = max(np.max(np.abs(n_i[x0,...])), ymax)
    ymin = min(0.0, ymin)

plt.xlabel(r'$y/d_i$')
plt.xlim(-20.0, 20.0)
plt.ylim(ymin, ymax)
plt.legend(loc='best')
plt.show()

In [ ]:
# Compute the reconnection rate
def compute_flux(pre, frame): 
    data_elc, data_ion, data_field = get_Gdata(pre, frame)
    r_coords, r_fluid_elc, r_fluid_ion, r_field = read_data(data_elc, data_ion, data_field)
    # Compute vector potential at X point.
    dx = r_coords[0][1] - r_coords[0][0]
    dy = r_coords[1][1] - r_coords[1][0]
    psi = np.zeros((Ny, Nx))
    psi = calc_psi2d(r_field[...,3].transpose(), r_field[...,4].transpose(), dx, dy)
    saddles = find_saddles(psi)
    psiX = psi[saddles[1][0], saddles[1][1]]
    psi0 = np.max(np.abs(psi))
    # Fetch Ez at X point
    EzX = r_field[saddles[1][1], saddles[1][0], 2]
    return psiX, psi0, EzX
    
num_frames = 26
psiX_10m = np.zeros(num_frames)
psi0_10m = np.zeros(num_frames)
EzX_10m = np.zeros(num_frames)
t = np.linspace(0, 50/omega_ci, num_frames)
rate_fac = 1/(B0*vA0)
for i in range(0, num_frames):
    # Compute the flux at the X-point in 10 moment simulation
    psiX_10m[i], psi0_10m[i], EzX_10m[i] = compute_flux("10m_magnetotail/10m_magnetotail", i) 

# We can either use Ez or the time derivative of the vector potential psi to estimate the reconnection rate. 
psi_diff_10m = np.abs(psi0_10m - psiX_10m)
dpsi_dt_10m = np.gradient(psi_diff_10m, t)
plt.figure()
plt.plot(t*omega_ci, np.abs(dpsi_dt_10m)*rate_fac, 'k-', label=r'$d\psi/dt$')
plt.plot(t*omega_ci, np.abs(EzX_10m)*rate_fac, 'k--', label=r'$E_z$')
plt.xlabel(r'$t \Omega_{ci}$')
plt.ylabel(r'$E_z/(v_A B_0)$')
plt.legend(loc='best')
plt.show()

## Dayside Reconnection (Magnetopause)

### Activity

Examine the dayside reconnection layer. Compare the features to previous cases, and identify differences.

In [ ]:
# Physical constants and derived parameters
epsilon0 = 1.0 # Permittivity of free space.
mu0 = 1.0 # Permeability of free space.
light_speed = 1.0/np.sqrt(epsilon0*mu0) # Speed of light. 
mass_ion = 1.0 # Ion mass.
charge_ion = 1.0 # Ion charge.
mass_elc = 1.0/100.0 # Electron mass.
charge_elc = -1.0 # Electron charge.
vAe = 0.2 # Electron Alfven velocity
vAi = vAe*np.sqrt(mass_elc/mass_ion)
n0 = 1.0 # Reference number density.
beta2 = 2.748 # Magnetosheath plasma beta
Ti1_over_Ti2 = 7.73/1.374 # Ion temperature / electron temperature.
Te1_over_Te2 = 1.288/1.374 # Ion temperature / electron temperature.
wpi = np.sqrt(charge_ion**2*n0/(epsilon0*mass_ion)) # Ion plasma frequency. 
wpe = np.sqrt(charge_ion**2*n0/(epsilon0*mass_elc)) # Electron plasma frequency. 
di = light_speed/wpi # Ion inertial length.
de = light_speed/wpe # Electron inertial length.
B0 = vAe*np.sqrt(n0*mass_elc) # Reference magnetic field strength. Sets ratio of vAe/c.
b1 = 1.696*B0
b2 = 1.0*B0
guide1 = 0.099*B0
guide2 = guide1
n1 = 0.062*n0
n2 = 1.0*n0
Ti2 = beta2*(b2*b2)/(2.0*n2*mu0)
Te1 = Ti2*Te1_over_Te2
Ti1 = Ti2*Ti1_over_Ti2
Te2 = (0.5*(b1*b1 - b2*b2) + 0.5*(guide1*guide1 - guide2*guide2) + n1*(Ti1 + Te1) - n2*Ti2)/n2
omega_ci = charge_ion * B0 / mass_ion # Ion cyclotron frequency. 

In [ ]:
# Frame of simulation data (0-100)
frame = 0

In [ ]:
# Load data - re-run if changing frame
data_elc, data_ion, data_field = get_Gdata("10m_magnetopause/10m_magnetopause", frame)
r_coords, r_fluid_elc, r_fluid_ion, r_field = read_data(data_elc, data_ion, data_field)

# Compute out-of-plane current density
Nx = r_fluid_ion.shape[0]
Ny = r_fluid_ion.shape[1]
Jz = np.zeros((Nx, Ny))
Jz = charge_ion/mass_ion*r_fluid_ion[...,3] + charge_elc/mass_elc*r_fluid_elc[...,3]
n_e = r_fluid_elc[...,0]
n_i = r_fluid_ion[...,0]
ux_e = r_fluid_elc[...,1]/r_fluid_elc[...,0]
uy_e = r_fluid_elc[...,2]/r_fluid_elc[...,0]
uz_e = r_fluid_elc[...,3]/r_fluid_elc[...,0]
ux_i = r_fluid_ion[...,1]/r_fluid_ion[...,0]
uy_i = r_fluid_ion[...,2]/r_fluid_ion[...,0]
uz_i = r_fluid_ion[...,3]/r_fluid_ion[...,0]
pxx = r_fluid_elc[:,:,4]
pxy = r_fluid_elc[:,:,5]
pxz = r_fluid_elc[:,:,6]
pyy = r_fluid_elc[:,:,7]
pyz = r_fluid_elc[:,:,8]
pzz = r_fluid_elc[:,:,9]

Pxx = pxx - n_e*ux_e*ux_e
Pxy = pxy - n_e*ux_e*uy_e
Pxz = pxz - n_e*ux_e*uz_e
Pyy = pyy - n_e*uy_e*uy_e
Pyz = pyz - n_e*uy_e*uz_e
Pzz = pzz - n_e*uz_e*uz_e

# Compute contours of the magnetic field.
Ex = r_field[...,0]
Ey = r_field[...,1]
Ez = r_field[...,2]
Bx = r_field[...,3]
By = r_field[...,4]
Bz = r_field[...,5]
dx = r_coords[0][1] - r_coords[0][0]
dy = r_coords[1][1] - r_coords[1][0]
psi = np.zeros((Ny, Nx))
psi = calc_psi2d(Bx.transpose(),By.transpose(), dx, dy)

# Compute magnitude of the magnetic field
_, magB_sq = pg.tools.mag_sq(data_field, '3:6')
magB = np.sqrt(magB_sq[...,0])

bx = Bx/magB
by = By/magB
bz = Bz/magB

# Compute local Alfven speed
_, rho_ion = pg.tools.get_density(data_ion)
vA = magB/np.sqrt(mu0*rho_ion[...,0])

# Compute normalized drift speed
coords, ux =  pg.tools.get_vx(data_ion)
coords, uy = pg.tools.get_vy(data_ion)
uxi_norm0 = ux[...,0]/vAi # Normalized x flow to upstream Alfven speed
uyi_norm0 = uy[...,0]/vAi

coords, ux =  pg.tools.get_vx(data_elc)
coords, uy = pg.tools.get_vy(data_elc)
uxe_norm0 = ux[...,0]/vAi # Normalized x flow to upstream Alfven speed
uye_norm0 = uy[...,0]/vAi

udrift = np.sqrt(ux_i**2 + uy_i**2)/vAi

# Compute X-point
saddles = find_saddles(psi)
xpoint = [coords[0][saddles[0][1]], coords[1][saddles[0][0]]]

# Calculate agyrotropy of the pressure tensor
Ppar = (Pxx*bx**2 + Pyy*by**2 + Pzz*bz**2 + 2.0*(Pxy*bx*by + Pxz*bx*bz + Pyz*by*bz))
Pperp = (Pxx + Pyy + Pzz - Ppar)/2.0
agyro = np.sqrt((Pxy**2 + Pxz**2 + Pyz**2)/(Pperp**2 + 2.0*Pperp*Ppar))

In [ ]:
# Reconnection layer

# Color plots - only activate one!
show_Jz = 0 # Out-of-plane current
show_Bz = 0 # Out-of-plane magnetic field
show_Ez = 0 # Out-of-plane electric field
show_magB = 0 # Magnetic field magnitude
show_vA = 0 # Alfven speed
show_ux = 0 # Ion parallel velocity
show_ion_drift = 1 # Ion bulk speed
show_n = 0 # Plasma density

# Streamlines
show_psi = 1 # Vector potential
show_ue = 0 # Electron drifts
show_ui = 0 # Ion drifts
show_xpoint = 0 # X-point

In [ ]:
# Plot quantities - re-run to plot new quantities
plt.figure(figsize=(10,2.5))
# Add shading='gouraud' for further interpolation and smoother plots
if show_Jz == True:
    vmax = np.max(np.abs(Jz))
    vmin = -vmax
    plt.pcolormesh(r_coords[0]/di, r_coords[1]/di, Jz.transpose(), vmax=vmax, vmin=vmin, cmap='seismic')#, shading='gouraud')
    plt.colorbar(format='%.2f', ticks=np.linspace(vmin, vmax, 5), fraction=0.046, pad=0.04)
    plt.title(r'$J_z$')
if show_Bz == True:
    vmax = np.max(np.abs(Bz))
    vmin = -vmax
    plt.pcolormesh(r_coords[0]/di, r_coords[1]/di, Bz.transpose(), vmax=vmax, vmin=vmin, cmap='seismic')#, shading='gouraud')
    plt.colorbar(format='%.2f', ticks=np.linspace(vmin, vmax, 5), fraction=0.046, pad=0.04)
    plt.title(r'$B_z$')
if show_Ez == True:
    vmax = np.max(np.abs(Ez))/10.0
    vmin = -vmax
    plt.pcolormesh(r_coords[0]/di, r_coords[1]/di, Ez.transpose(), vmax=vmax, vmin=vmin, cmap='seismic')#, shading='gouraud')
    plt.colorbar(format='%.2f', ticks=np.linspace(vmin, vmax, 5), fraction=0.046, pad=0.04)
    plt.title(r'$E_z$')
if show_magB == True:
    vmax = np.max(np.abs(magB))
    vmin = 0.0
    plt.pcolormesh(r_coords[0]/di, r_coords[1]/di, magB.transpose(), vmax=vmax, vmin=vmin, cmap='inferno')#, shading='gouraud')
    plt.colorbar(format='%.2f', ticks=np.linspace(vmin, vmax, 5), fraction=0.046, pad=0.04)
    plt.title(r'$|\mathbf{B}|$')
if show_vA == True:
    vmax = np.max(np.abs(vA))
    vmin = 0.0
    plt.pcolormesh(r_coords[0]/di, r_coords[1]/di, vA.transpose(), vmax=vmax, vmin=vmin, cmap='inferno')#, shading='gouraud')
    plt.colorbar(format='%.1f', ticks=np.linspace(vmin, vmax, 5), fraction=0.046, pad=0.04)
    plt.title(r'$v_A$')
if show_ux == True:
    vmax = np.max(np.abs(uxi_norm0))
    vmin = -vmax
    plt.pcolormesh(r_coords[0]/di, r_coords[1]/di, uxi_norm0.transpose(), vmax=vmax, vmin=vmin, cmap='seismic')#, shading='gouraud')
    plt.colorbar(format='%.1f', ticks=np.linspace(vmin, vmax, 5), fraction=0.046, pad=0.04)
    plt.title(r'$u_x/v_A$')
if show_ion_drift == True:
    vmax = np.max(np.abs(udrift))
    vmin = 0.0
    plt.pcolormesh(r_coords[0]/di, r_coords[1]/di, udrift.transpose(), vmax=vmax, vmin=vmin, cmap='inferno')#, shading='gouraud')
    plt.colorbar(format='%.1f', ticks=np.linspace(vmin, vmax, 5), fraction=0.046, pad=0.04)
    plt.title(r'$u/v_A$')
if show_n == True:
    vmax = np.max(np.abs(n_e + n_i))
    vmin = 0.0
    plt.pcolormesh(r_coords[0]/di, r_coords[1]/di, (n_e + n_i).transpose(), vmax=vmax, vmin=vmin, cmap='inferno')#, shading='gouraud')
    plt.colorbar(format='%.1f', ticks=np.linspace(vmin, vmax, 5), fraction=0.046, pad=0.04)
    plt.title(r'$n$')
    
X, Y = np.meshgrid(r_coords[0]/di, r_coords[1]/di)
if show_psi == True:
    plt.contour(r_coords[0]/di, r_coords[1]/di, psi, 7, colors="k", linestyles="solid")
if show_ue == True:
    plt.streamplot(X, Y, ux_e.transpose(), uy_e.transpose(), color='green', density=1.0)
if show_ui == True:
    plt.streamplot(X, Y, ux_i.transpose(), uy_i.transpose(), color='purple', density=0.7, broken_streamlines=True)

if show_xpoint == True:
    plt.scatter(xpoint[0]/di, xpoint[1]/di, color='green', marker='X')
    
plt.xlabel(r'$x/d_i$')
plt.ylabel(r'$y/d_i$')
plt.ylim([0.0, 10.24])
plt.setp(plt.gca(), aspect=1.0)
plt.show()

### Discussion
- How are the initial conditions different from previous cases?
- Which side is the magnetosheath? Which side is the magnetosphere?
- What gradients form across the layer?
- How does the asymmetry change the ion inflow stagnation point?
- How many poles does the out-of-plane magnetic field have? Is this different from the symmetric case?
- How does being 2D limit this simulation?

In [ ]:
# Profiles across the reconnection layer
show_Bx = 1
show_Jz = 1
show_uy = 1
show_ne = 0
show_ni = 0

In [ ]:
# Plot at x/di=20.48
x0 = int(Nx/2)
plt.figure()
ymax = 0.0
ymin = 0.0
if show_Bx == True:
    plt.plot(r_coords[1]/di, Bx[x0,...], 'k', label=r'$B_x$')
    ymax = max(np.max(np.abs(Bx[x0,...])), ymax)
    ymin = -ymax
if show_Jz == True:
    plt.plot(r_coords[1]/di, Jz[x0,...], 'b', label=r'$J_z$')
    ymax = max(np.max(np.abs(Jz[x0,...])), ymax)
    ymin = -ymax
if show_uy == True:
    plt.plot(r_coords[1]/di, uy_norm0[x0,...], 'r', label=r'$u_y/v_A$')
    ymax = max(np.max(np.abs(uy_norm0[x0,...])), ymax)
    ymin = -ymax
if show_ne == True:
    plt.plot(r_coords[1]/di, n_e[x0,...], 'y', label=r'$n_e$')
    ymax = max(np.max(np.abs(n_e[x0,...])), ymax)
    ymin = min(0.0, ymin)
if show_ni == True:
    plt.plot(r_coords[1]/di, n_i[x0,...], 'r', label=r'$n_i$')
    ymax = max(np.max(np.abs(n_i[x0,...])), ymax)
    ymin = min(0.0, ymin)

plt.xlabel(r'$y/d_i$')
plt.xlim(0.0, 10.24)
plt.ylim(ymin, ymax)
plt.legend(loc='best')
plt.show()

In [ ]:
# Compute the reconnection rate
def compute_flux(pre, frame): 
    data_elc, data_ion, data_field = get_Gdata(pre, frame)
    r_coords, r_fluid_elc, r_fluid_ion, r_field = read_data(data_elc, data_ion, data_field)
    # Compute vector potential at X point.
    dx = r_coords[0][1] - r_coords[0][0]
    dy = r_coords[1][1] - r_coords[1][0]
    psi = np.zeros((Ny, Nx))
    psi = calc_psi2d(r_field[...,3].transpose(), r_field[...,4].transpose(), dx, dy)
    saddles = find_saddles(psi)
    psiX = psi[saddles[0][0], saddles[0][1]]
    psi0 = np.min(np.abs(psi))
    # Fetch Ez at X point
    EzX = r_field[saddles[0][1], saddles[0][0], 2]
    return psiX, psi0, EzX
    
num_frames = 61
psiX_10m = np.zeros(num_frames)
psi0_10m = np.zeros(num_frames)
EzX_10m = np.zeros(num_frames)
t = np.linspace(0, 50/omega_ci, num_frames)
rate_fac = 1/(B0*vAi)
for i in range(0, num_frames):
    # Compute the flux at the X-point in 10 moment simulation
    psiX_10m[i], psi0_10m[i], EzX_10m[i] = compute_flux("10m_magnetopause/10m_magnetopause", i)

# We can either use Ez or the time derivative of the vector potential psi to estimate the reconnection rate. 
psi_diff_10m = np.abs(psi0_10m - psiX_10m)
dpsi_dt_10m = np.gradient(psi_diff_10m, t)
plt.figure()
plt.plot(t*omega_ci, np.abs(dpsi_dt_10m)*rate_fac, 'k-', label=r'$d\psi/dt$')
plt.plot(t*omega_ci, np.abs(EzX_10m)*rate_fac, 'k--', label=r'$E_z$')
plt.xlabel(r'$t \Omega_{ci}$')
plt.ylabel(r'$E_z/(v_A B_0)$')
plt.legend(loc='best')
plt.show()

## Part 3: In-Situ Measurement and Identification of Reconnection

We have taken a comprehensive look at the different characteristics of reconnection, but what does reconnection look like to a spacecraft that cannot measure and map out the entire layer?

### Activity

Ten random spacecraft trajectories have been generated. Using an ID from 0-9, the following script will plot data sampled along that trajectory. In groups, analyze the data associated with your assigned ID and determine:
1. Is this magnetopause or magnetotail reconnection?
2. Where is the X-point?
3. What is the approximate trajectory the spacecraft is taking through the reconnection layer?

In [ ]:
# ID number for spacecraft data
group_id = 0

In [ ]:
# Plot spacecraft data
rd_file = "sc_data/random_%d.npz"
sc_data = np.load(rd_file % group_id)

fig, ax = plt.subplots(5, sharex=True, figsize=(10,7))
ax[0].plot(sc_data['t'], sc_data['bx'], label="$B_x$")
ax[0].plot(sc_data['t'], sc_data['by'], label="$B_y$")
ax[0].plot(sc_data['t'], sc_data['bz'], label="$B_z$")
ax[0].axhline(0.0, sc_data['t'][0], sc_data['t'][-1], ls='--', color='k')
ax[0].set_ylabel("$B_{xyz}$")
ax[0].legend(bbox_to_anchor=(1.0, 1), loc="upper left")
ax[1].plot(sc_data['t'], sc_data['n'])
ax[1].axhline(0.0, sc_data['t'][0], sc_data['t'][-1], ls='--', color='k')
ax[1].set_ylabel("$n$")
ax[2].plot(sc_data['t'], sc_data['ux'], label="$u_x$")
ax[2].plot(sc_data['t'], sc_data['uy'], label="$u_y$")
ax[2].axhline(0.0, sc_data['t'][0], sc_data['t'][-1], ls='--', color='k')
ax[2].legend(bbox_to_anchor=(1.0, 1), loc="upper left")
ax[2].set_ylabel("$u_{i,xyz}$")
ax[3].plot(sc_data['t'], sc_data['jz'])
ax[3].axhline(0.0, sc_data['t'][0], sc_data['t'][-1], ls='--', color='k')
ax[3].set_ylabel("$J_z$")
ax[4].plot(sc_data['t'], sc_data['ex'], label="$E_x$")
ax[4].plot(sc_data['t'], sc_data['ez'], label="$E_y$")
ax[4].plot(sc_data['t'], sc_data['ey'], label="$E_z$")
ax[4].axhline(0.0, sc_data['t'][0], sc_data['t'][-1], ls='--', color='k')
ax[4].set_ylabel("$E_{xyz}$")
ax[4].legend(bbox_to_anchor=(1.0, 1), loc="upper left")
plt.subplots_adjust(wspace=0, hspace=0)
plt.xlim([0.0, sc_data['t'][-1]])
plt.xlabel('Time (a.u.)')

### Discussion

How might you estimate the reconnection rate from the spacecraft data? How well does that estimate compare to the calculations from Part 2?

In [ ]:
%stop_runtime test_cpu
%switch_runtime local